# Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

# Number of LAPPD events

In [ ]:
## Create a list of runs and the associated LAPPD event numbers

# read in file of run numbers
runs = []
with open('runlists/runlist_r3844-r3649.txt','r') as f:
    for run in f:
        runs.append(int(run))
f.close()

#Set up storing containers for event numbers
events = []
total_events = []

#Set up column names
col_names = ["Filename", "VME events", "MRD events", "Trig events", "LAPPD events"]

#Loop through runs, extract the number of total LAPPD events per run
global_counter=0
for run in runs:
    print(run)
    str_filename = "NumEvents/NumEvents_R"+str(int(run))+".csv"
    df = pd.read_csv(str_filename, sep=',', header=None, names=col_names)
    sum = 0
    counter=0
    for x in df["LAPPD events"]:
        if (x != ' '):
            #print(counter, x, sum)
            sum += int(x)
            counter += 1
    print('Run',run,': LAPPD events =',sum)
    global_counter += sum
    events.append(sum)
    total_events.append(global_counter)

#Total event statistics
print("#### LAPPD event statistics #####")
print("Global event number:",global_counter)
print("#### End #####")

#Plot event statistics vs run
#plt.plot(runs,events,'b*')
#plt.xlabel("Run")
#plt.ylabel("LAPPD events")

#Plot cumulative event statistics vs run
plt.plot(runs,total_events,'r-')
plt.grid()
plt.xlabel("Run")
plt.ylabel("Cumulative LAPPD events")

# Filesizes (VME bug)

In [ ]:
## Create a list of runs and the associated file sizes (in kB)

#Set up column names
col_names = ["Run", "SubRun", "Part", "Filesize"]

# read in file sizes file
df = pd.read_csv('filesizes.txt', sep=',', header=None, names=col_names)

df.head()

plt.plot(df["Run"]+0.001*df["Part"],df["Filesize"],'.k')
plt.xlabel('Run')
plt.ylabel('Filesize [kB]')
 
counter=0
    
for x in range(len(df["Filesize"])):
    if (int(df["Filesize"][x])<10):
        print(df["Run"][x],",",df["Part"][x])
        if df["Run"][x] == 3833 or df["Run"][x]!= 3798:
            counter+=1

print('Failed event building: ',counter,', total files: ',len(df["Filesize"]))

# Detailed event statistics

In [ ]:
## Create a list of runs and count the number of events...
# ... in the beam window
# ... with a MRD track
# ... with no veto activity

# read in file of run numbers
runs = []
#with open('runlist_r3760-r3696_lappd.txt','r') as f:
with open('runlists/runlist_r3844-r3649.txt','r') as f:

    for run in f:
        runs.append(int(run))
f.close()

#runs.append(3756)
#runs.append(3754)
print(runs)

#Set up list for beamgate timing plot
list_beamgate_timing = []

#Set up column names
col_names = ["Run", "SubRun", "Part", "Ev", "GlobalEv","TriggerTime","LAPPDEventTime","LAPPDOffset","TimeSinceBeamgate","InBeamWindow","MRDTrack","NoVeto","Dummy"]

list_run = []
list_events = []
list_total_events = []
list_beam = []
list_total_beam = []
list_beam_mrd = []
list_total_beam_mrd = []
list_beam_mrd_noveto = []
list_total_beam_mrd_noveto = []

fout = open('LAPPDEventStatistics/LAPPDEventStatistics_BeamMRDNoVeto_R3649-R3750.csv','w')

#Loop through runs, extract the number of total LAPPD events per run
global_counter=0
global_beam=0
global_beam_mrd=0
global_beam_mrd_noveto=0
for run in runs:
    
    #Read in file
    str_filename = "LAPPDStatistics/LAPPDStatistics_R"+str(int(run))+".csv"
    print(str_filename)
    df = pd.read_csv(str_filename, sep=',', header=None, names=col_names)
    #df = pd.read_csv(str_filename, header = 0)
    
    #Initialize variables / event counters
    sum_inbeam = 0
    sum_mrd = 0
    sum_noveto = 0
    sum_inbeam_mrd = 0
    sum_inbeam_mrd_noveto = 0
    counter=0
    
    df.head()
    
    #print some contents of the data frame, increase counters
    for x in df["TimeSinceBeamgate"]:
        #list_beamgate_timing.append(x)
        if int(x) < 0.5E16:
            list_beamgate_timing.append(x)
       # print(x)
       # if x > 0.5E16:
       #     print(x)

    for x in df["InBeamWindow"]:
       # print(x)
        if int(x) == 1:
            #print(counter, x, sum)
            sum_inbeam += 1
            global_beam += 1
        counter += 1
    for x in df["MRDTrack"]:
       # print(x)
        if int(x) == 1:
            sum_mrd += 1
    for x in df["NoVeto"]:
        if int(x) == 1:
            sum_noveto += 1
    for i in range(len(df["InBeamWindow"])):
        if int(df["InBeamWindow"][i])==1 and int(df["MRDTrack"][i])==1:
            sum_inbeam_mrd += 1
            global_beam_mrd += 1
            if (int(df["NoVeto"][i])==1):
                sum_inbeam_mrd_noveto +=1
                global_beam_mrd_noveto += 1
    
    #print('Run',run,': events =',counter,' in_beam =',sum_inbeam, ', mrdtrack = ',sum_mrd, ', noveto = ',sum_noveto, ', beam & mrd = ',sum_inbeam_mrd, ', beam & mrd & noveto = ',sum_inbeam_mrd_noveto)
    global_counter += counter
    list_run.append(run)
    list_events.append(counter)
    list_total_events.append(global_counter)
    list_beam.append(sum_inbeam)
    list_beam_mrd.append(sum_inbeam_mrd)
    list_beam_mrd_noveto.append(sum_inbeam_mrd_noveto)
    list_total_beam.append(global_beam)
    list_total_beam_mrd.append(global_beam_mrd)
    list_total_beam_mrd_noveto.append(global_beam_mrd_noveto)
    str_out = str(run) + "," + str(counter) + "," + str(sum_inbeam) + "," + str(sum_inbeam_mrd) + ","+ str(sum_inbeam_mrd_noveto)
    #print(str_out)
    fout.write(str_out)
    fout.write('\n')

#Total event statistics
print("#### LAPPD event statistics #####")
print("Global event number:",global_counter)
print("#### End #####")

#Plot beamgate timing histogram
plt.hist(list_beamgate_timing,bins=160,range=[0, 20000])
plt.xlabel('Time since start of beamgate [ns]', size=16)
plt.ylabel('Counts',size = 16)
plt.savefig('plots/TimeSinceBeamgate_R3844-R3649.pdf')

In [ ]:
#Plot beamgate timing histogram
plt.hist(list_beamgate_timing,bins=128,range=[7000, 10000])
plt.xlabel('Time since start of beamgate [ns]', size=16)
plt.ylabel('Counts',size = 16)
plt.savefig('plots/TimeSinceBeamgate_R3844-R3649_zoom.pdf')

In [ ]:
plt.plot(runs,total_events,'r-',label='raw events')
plt.grid()
plt.xlabel("Run")
plt.ylabel("Combined LAPPD events")

plt.plot(runs,list_total_events,'b-',label='processed events')
plt.legend()
plt.savefig('plots/LAPPD_TotalEvents_Raw_Processed.pdf')

In [ ]:

plt.plot(runs,list_total_events,'b-',label='all processed')
plt.grid()
plt.xlabel("Run")
plt.ylabel("Combined LAPPD events")

plt.plot(runs,list_total_beam,'k-',label='beam window')
plt.plot(runs,list_total_beam_mrd,'r-',label='beam + MRD')
plt.plot(runs,list_total_beam_mrd_noveto,'g-',label='beam + MRD + noveto')
plt.legend()
plt.yscale("log")
plt.savefig('plots/LAPPD_TotalEvents_DifferentCuts.pdf')

In [ ]:
list_frac_all = []
list_frac_beam = []
list_frac_beam_mrd = []
list_frac_beam_mrd_noveto = []
for i in range(len(list_total_beam)):
    list_frac_all.append(1)
    list_frac_beam.append(list_total_beam[i]/list_total_events[i])
    list_frac_beam_mrd.append(list_total_beam_mrd[i]/list_total_events[i])
    list_frac_beam_mrd_noveto.append(list_total_beam_mrd_noveto[i]/list_total_events[i])



plt.plot(runs,list_frac_all,'b-',label='all processed')
plt.grid()
plt.xlabel("Run")
plt.ylabel("Combined Fraction")
plt.plot(runs,list_frac_beam,'k-',label='beam window')
plt.plot(runs,list_frac_beam_mrd,'r-',label='beam + MRD')
plt.plot(runs,list_frac_beam_mrd_noveto,'g-',label='beam + MRD + noveto')
plt.legend()
plt.yscale("log")
plt.savefig('plots/LAPPD_TotalFractions_DifferentCuts.pdf')

In [ ]:
list_frac_All = []
list_frac_Beam = []
list_frac_Beam_mrd = []
list_frac_Beam_mrd_noveto = []
for i in range(len(list_beam)):
    list_frac_All.append(1)
    if list_events[i]>0:
        list_frac_Beam.append(list_beam[i]/list_events[i])
        list_frac_Beam_mrd.append(list_beam_mrd[i]/list_events[i])
        list_frac_Beam_mrd_noveto.append(list_beam_mrd_noveto[i]/list_events[i])
    else:
        list_frac_Beam.append(0)
        list_frac_Beam_mrd.append(0)
        list_frac_Beam_mrd_noveto.append(0)
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

plt.plot(runs,list_frac_All,'bs',label='all processed')
plt.grid()
plt.xlabel("Run")
plt.ylabel("Fraction")
plt.plot(runs,list_frac_Beam,'ks',label='beam window')
plt.plot(runs,list_frac_Beam_mrd,'rs',label='beam + MRD')
plt.plot(runs,list_frac_Beam_mrd_noveto,'gs',label='beam + MRD + noveto')
plt.legend()
plt.yscale("log")
plt.savefig('plots/LAPPD_Fractions_DifferentCuts.pdf')

In [ ]:
plt.plot(runs,events,'r--',label='raw events')
plt.grid()
plt.xlabel("Run")
plt.ylabel("LAPPD events")

plt.plot(runs,list_events,'b-',label='processed events')
plt.legend()
plt.savefig('plots/LAPPD_Events_Raw_Processed.pdf')

In [ ]:
list_ratio_processed=[]
runs_processed=[]
for raw,processed,run  in zip(events,list_events,runs):
    if raw >0:
        list_ratio_processed.append(processed/raw)
        runs_processed.append(run)
        print('Run: ',run,', raw:',raw,'processed:',processed)
    else:
        print('Run ',run,'has 0 raw data entries. Skip!')
plt.plot(runs_processed,list_ratio_processed,'ko')
plt.xlabel('Run')
plt.ylabel('Fraction processed/raw events')
plt.savefig('plots/LAPPD_Events_Fraction_Processed.pdf')

In [ ]:
list_ratio_processed_total=[]
runs_processed_total=[]
for raw,processed,run  in zip(total_events,list_total_events,runs):
    if raw >0:
        list_ratio_processed_total.append(processed/raw)
        runs_processed_total.append(run)
    else:
        print('Run ',run,'has 0 raw data entries. Skip!')
plt.plot(runs_processed_total,list_ratio_processed_total,'r-')
plt.xlabel('Run')
plt.ylabel('Fraction processed/raw events (combined)')
plt.savefig('plots/LAPPD_Events_Fraction_Processed_Combined.pdf')

# Stability of time offset

In [ ]:
# Set up lists / containers
list_triggertime = []
list_lappd_offset = []
list_triggertime_datetime = []
list_lappd_offset_datetime = []
list_runs = []
list_subruns = []
list_parts = []


# read in file of run numbers
runs = []
#with open('runlist_r3760-r3696_lappd.txt','r') as f:
with open('runlists/runlist_r3844-r3649.txt','r') as f:
    for run in f:
        runs.append(int(run))
f.close()

#runs.append(3756)
#runs.append(3754)
print(runs)

#Set up column names
col_names = ["Run", "SubRun", "Part", "Ev", "GlobalEv","TriggerTime","LAPPDEventTime","LAPPDOffset","TimeSinceBeamgate","InBeamWindow","MRDTrack","NoVeto","Dummy"]

#Loop through runs, extract the number of total LAPPD events per run
global_counter=0
for run in runs:
    if run == 3713: 
        continue
    print(run)
    
    #Read in file
    str_filename = "LAPPDStatistics/LAPPDStatistics_R"+str(int(run))+".csv"
    print(str_filename)
    df = pd.read_csv(str_filename, sep=',', header=None, names=col_names)
    for x in df["TriggerTime"]:
        list_triggertime.append(int(x)/1E9)
        list_triggertime_datetime.append(dt.datetime.fromtimestamp(int(x)/1E9))
        #print(x)
    for x in df["LAPPDOffset"]:
        #print(x)
        list_lappd_offset.append(int(x)/1E9)
        list_lappd_offset_datetime.append(dt.datetime.fromtimestamp(int(x)/1E9))
    for x in df["Run"]:
        list_runs.append(int(x))
    for x in df["SubRun"]:
        list_subruns.append(int(x))
    for x in df["Part"]:
        list_parts.append(int(x))
        #print('Append part ',x)

fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)
        
#plt.plot(list_triggertime_datetime,list_lappd_offset_datetime,'x')
#plt.xlabel('TriggerTime',size=16)
#plt.ylabel('LAPPD offset',size=16)
#plt.xticks(rotation=45)
#plt.savefig("LAPPDOffset_Triggertime_Datetime_R3649-R3760.pdf")

plt.plot(list_triggertime_datetime,list_lappd_offset,'x')
plt.xlabel('TriggerTime',size=16)
plt.ylabel('LAPPD offset [s]',size=16)
plt.xticks(rotation=45)
plt.savefig("plots/LAPPDOffset_Triggertime_R3844-R3649.pdf")

#plt.hist(list_lappd_offset_datetime)
#plt.xlabel('LAPPD offset')
#plt.ylabel('#')
#plt.savefig("LAPPDOffset_1D.pdf")

In [ ]:
plt.hist(list_lappd_offset, bins=1000, range = (1653161000,1653161025))
plt.xlabel('LAPPD offset [s]')
plt.ylabel('Counts')
plt.savefig("plots/LAPPDOffset_1D.pdf")

In [ ]:
list_triggertime_subset=[]
list_lappd_offset_subset=[]
list_triggertime_datetime_subset=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1653161025:
        list_triggertime_subset.append(list_triggertime[x])
        list_lappd_offset_subset.append(list_lappd_offset[x])
        list_triggertime_datetime_subset.append(dt.datetime.fromtimestamp(list_triggertime[x]))


# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff = np.polyfit(list_triggertime_subset, list_lappd_offset_subset, 1)
print(coeff)
poly1d_fn = np.poly1d(coeff)


plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset,poly1d_fn(list_triggertime_subset),'--k')
plt.ylim([1653161000,1653161025])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area1.pdf')

list_drift=[]
timestamp_drift=[]
list_drift.append(coeff[0])
timestamp_drift.append(dt.datetime.strptime('01/06/2022 18:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
list_triggertime_subset2=[]
list_lappd_offset_subset2=[]
list_triggertime_datetime_subset2=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1654536000 and list_lappd_offset[x] > 1654535980:
        list_triggertime_subset2.append(list_triggertime[x])
        list_lappd_offset_subset2.append(list_lappd_offset[x])
        list_triggertime_datetime_subset2.append(dt.datetime.fromtimestamp(list_triggertime[x]))


# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff2 = np.polyfit(list_triggertime_subset2, list_lappd_offset_subset2, 1)
print(coeff2)
poly1d_fn2 = np.poly1d(coeff2)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset2,poly1d_fn2(list_triggertime_subset2),'--k')
plt.ylim([1654535987,1654535988])
plt.xlim([dt.datetime.fromtimestamp(1654546000),dt.datetime.fromtimestamp(1654568000)])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area2.pdf')

list_drift.append(coeff2[0])
timestamp_drift.append(dt.datetime.strptime('06/06/2022 18:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
list_triggertime_subset3=[]
list_lappd_offset_subset3=[]
list_triggertime_datetime_subset3=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1654585902.05 and list_lappd_offset[x] > 1654585901:
        list_triggertime_subset3.append(list_triggertime[x])
        list_lappd_offset_subset3.append(list_lappd_offset[x])
        list_triggertime_datetime_subset3.append(dt.datetime.fromtimestamp(list_triggertime[x]))


# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff3= np.polyfit(list_triggertime_subset3, list_lappd_offset_subset3, 1)
print(coeff3)
poly1d_fn3 = np.poly1d(coeff3)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset3,poly1d_fn3(list_triggertime_subset3),'--k')
plt.ylim([1654585901.975,1654585902.05])
plt.xlim([dt.datetime.fromtimestamp(1654616000),dt.datetime.fromtimestamp(1654624000)])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area3.pdf')

list_drift.append(coeff3[0])
timestamp_drift.append(dt.datetime.strptime('07/06/2022 18:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
list_triggertime_subset4=[]
list_lappd_offset_subset4=[]
list_triggertime_datetime_subset4=[]
list_runs_subset4=[]
list_parts_subset4=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1654669361.05 and list_lappd_offset[x] > 1654669337.975:
        list_triggertime_subset4.append(list_triggertime[x])
        list_lappd_offset_subset4.append(list_lappd_offset[x])
        list_triggertime_datetime_subset4.append(dt.datetime.fromtimestamp(list_triggertime[x]))
        list_runs_subset4.append(list_runs[x])
        list_parts_subset4.append(list_parts[x])


# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff4= np.polyfit(list_triggertime_subset4, list_lappd_offset_subset4, 1)
print(coeff4)
poly1d_fn4 = np.poly1d(coeff4)

format_coeff4 = "{:.20f}".format(coeff4[0])
format_coeff4_2 = "{:.20f}".format(coeff4[1])


print(format_coeff4)
print(format_coeff4_2)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset4,poly1d_fn4(list_triggertime_subset4),'--k')
plt.ylim([1654669337.975,1654669345.05])
plt.xlim([dt.datetime.fromtimestamp(1654629000),dt.datetime.fromtimestamp(1656339000)])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area4.pdf')

list_drift.append(coeff4[0])
timestamp_drift.append(dt.datetime.strptime('15/06/2022 18:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
list_triggertime_subset5=[]
list_lappd_offset_subset5=[]
list_triggertime_datetime_subset5=[]
list_runs_subset5=[]
list_parts_subset5=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1655954605.05 and list_lappd_offset[x] > 1655954598.05:
        list_triggertime_subset5.append(list_triggertime[x])
        list_lappd_offset_subset5.append(list_lappd_offset[x])
        list_triggertime_datetime_subset5.append(dt.datetime.fromtimestamp(list_triggertime[x]))
        list_runs_subset5.append(list_runs[x])
        list_parts_subset5.append(list_parts[x])


# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff5= np.polyfit(list_triggertime_subset5, list_lappd_offset_subset5, 1)
print(coeff5)
poly1d_fn5 = np.poly1d(coeff5)

format_coeff5 = "{:.20f}".format(coeff5[0])
format_coeff5_2 = "{:.20f}".format(coeff5[1])


print(format_coeff5)
print(format_coeff5_2)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset5,poly1d_fn5(list_triggertime_subset5),'--k')
plt.ylim([1655954599.975,1655954605.05])
plt.xlim([dt.datetime.strptime('22/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S'),dt.datetime.strptime('26/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S')])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area5.pdf')

list_drift.append(coeff5[0])
timestamp_drift.append(dt.datetime.strptime('24/06/2022 18:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x')
plt.ylim([1656154593.975,1656454605.05])
plt.xlim([dt.datetime.strptime('22/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S'),dt.datetime.strptime('13/07/2022 00:00:00', '%d/%m/%Y %H:%M:%S')])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_AreaLateZoom.pdf')

In [ ]:
list_triggertime_subset6=[]
list_lappd_offset_subset6=[]
list_triggertime_datetime_subset6=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1656189110.05 and list_lappd_offset[x] > 1656189108.975:
        list_triggertime_subset6.append(list_triggertime[x])
        list_lappd_offset_subset6.append(list_lappd_offset[x])
        list_triggertime_datetime_subset6.append(dt.datetime.fromtimestamp(list_triggertime[x]))


# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff6= np.polyfit(list_triggertime_subset6, list_lappd_offset_subset6, 1)
print(coeff6)
poly1d_fn6 = np.poly1d(coeff6)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset6,poly1d_fn6(list_triggertime_subset6),'--k')
plt.ylim([1656189108.975,1656189110.05])
plt.xlim([dt.datetime.strptime('25/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S'),dt.datetime.strptime('27/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S')])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area6.pdf')

list_drift.append(coeff6[0])
timestamp_drift.append(dt.datetime.strptime('26/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
list_triggertime_subset7=[]
list_lappd_offset_subset7=[]
list_triggertime_datetime_subset7=[]
list_runs_subset7=[]
list_parts_subset7=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1656258920.05 and list_lappd_offset[x] > 1656258900.975:
        list_triggertime_subset7.append(list_triggertime[x])
        list_lappd_offset_subset7.append(list_lappd_offset[x])
        list_triggertime_datetime_subset7.append(dt.datetime.fromtimestamp(list_triggertime[x]))
        list_runs_subset7.append(list_runs[x])
        list_parts_subset7.append(list_parts[x])


# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff7= np.polyfit(list_triggertime_subset7, list_lappd_offset_subset7, 1)
print(coeff7)
poly1d_fn7 = np.poly1d(coeff7)
print(coeff7[0])

format_coeff7 = "{:.20f}".format(coeff7[0])
format_coeff7_2 = "{:.20f}".format(coeff7[1])
print(format_coeff7)
print(format_coeff7_2)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset7,poly1d_fn7(list_triggertime_subset7),'--k')
plt.ylim([1656258910.575,1656258912.55])
plt.xlim([dt.datetime.strptime('26/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S'),dt.datetime.strptime('28/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S')])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area7.pdf')

list_drift.append(coeff7[0])
timestamp_drift.append(dt.datetime.strptime('27/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
list_triggertime_subset8=[]
list_lappd_offset_subset8=[]
list_triggertime_datetime_subset8=[]
list_runs_subset8=[]
list_parts_subset8=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1656313378.05 and list_lappd_offset[x] > 1656313370.975:
        list_triggertime_subset8.append(list_triggertime[x])
        list_lappd_offset_subset8.append(list_lappd_offset[x])
        list_triggertime_datetime_subset8.append(dt.datetime.fromtimestamp(list_triggertime[x]))
        list_runs_subset8.append(list_runs[x])
        list_parts_subset8.append(list_parts[x])

# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff8= np.polyfit(list_triggertime_subset8, list_lappd_offset_subset8, 1)
print(coeff8)
format_coeff8 = "{:.20f}".format(coeff8[0])
format_coeff8_2 = "{:.20f}".format(coeff8[1])


print(format_coeff8)
print(format_coeff8_2)


poly1d_fn8 = np.poly1d(coeff8)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset8,poly1d_fn8(list_triggertime_subset8),'--k')
plt.ylim([1656313376.8,1656313377.2])
plt.xlim([dt.datetime.strptime('27/06/2022 10:00:00', '%d/%m/%Y %H:%M:%S'),dt.datetime.strptime('27/06/2022 18:00:00', '%d/%m/%Y %H:%M:%S')])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area8.pdf')

list_drift.append(coeff8[0])
timestamp_drift.append(dt.datetime.strptime('27/06/2022 13:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
list_triggertime_subset9=[]
list_lappd_offset_subset9=[]
list_triggertime_datetime_subset9=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1656339361.75 and list_lappd_offset[x] > 1656339360.975:
        list_triggertime_subset9.append(list_triggertime[x])
        list_lappd_offset_subset9.append(list_lappd_offset[x])
        list_triggertime_datetime_subset9.append(dt.datetime.fromtimestamp(list_triggertime[x]))


# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff9= np.polyfit(list_triggertime_subset9, list_lappd_offset_subset9, 1)
print(coeff9)
poly1d_fn9 = np.poly1d(coeff9)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset9,poly1d_fn9(list_triggertime_subset9),'--k')
plt.ylim([1656339360.975,1656339361.75])
plt.xlim([dt.datetime.strptime('27/06/2022 16:00:00', '%d/%m/%Y %H:%M:%S'),dt.datetime.strptime('28/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S')])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area9.pdf')

list_drift.append(coeff9[0])
timestamp_drift.append(dt.datetime.strptime('27/06/2022 18:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
list_triggertime_subset10=[]
list_lappd_offset_subset10=[]
list_triggertime_datetime_subset10=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1656361089.25 and list_lappd_offset[x] > 1656361089.17:
        list_triggertime_subset10.append(list_triggertime[x])
        list_lappd_offset_subset10.append(list_lappd_offset[x])
        list_triggertime_datetime_subset10.append(dt.datetime.fromtimestamp(list_triggertime[x]))


# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff10= np.polyfit(list_triggertime_subset10, list_lappd_offset_subset10, 1)
print(coeff10)
poly1d_fn10 = np.poly1d(coeff10)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset10,poly1d_fn10(list_triggertime_subset10),'--k')
plt.ylim([1656361089.17,1656361089.25])
plt.xlim([dt.datetime.strptime('27/06/2022 22:00:00', '%d/%m/%Y %H:%M:%S'),dt.datetime.strptime('28/06/2022 00:00:00', '%d/%m/%Y %H:%M:%S')])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area10.pdf')

list_drift.append(coeff10[0])
timestamp_drift.append(dt.datetime.strptime('27/06/2022 23:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
list_triggertime_subset11=[]
list_lappd_offset_subset11=[]
list_triggertime_datetime_subset11=[]
list_runs_subset11=[]
list_parts_subset11=[]
print(len(list_runs),', ',len(list_parts))
for x in range(len(list_runs)):
    if list_lappd_offset[x] < 1656381130.25 and list_lappd_offset[x] > 1656381065.17:
        list_triggertime_subset11.append(list_triggertime[x])
        list_lappd_offset_subset11.append(list_lappd_offset[x])
        list_triggertime_datetime_subset11.append(dt.datetime.fromtimestamp(list_triggertime[x]))
        list_runs_subset11.append(list_runs[x])
        list_parts_subset11.append(list_parts[x])


# Create figure
        
fig = plt.figure() 

# Figure size

fig.set_size_inches(12,10)

coeff11= np.polyfit(list_triggertime_subset11, list_lappd_offset_subset11, 1)
print(coeff11)
poly1d_fn11 = np.poly1d(coeff11)

format_coeff11 = "{:.20f}".format(coeff11[0])
format_coeff11_2 = "{:.20f}".format(coeff11[1])


print(format_coeff11)
print(format_coeff11_2)

plt.plot(list_triggertime_datetime,list_lappd_offset,'x',list_triggertime_datetime_subset11,poly1d_fn11(list_triggertime_subset11),'--k')
plt.ylim([1656381085.17,1656381095.25])
plt.xlim([dt.datetime.strptime('26/06/2022 22:00:00', '%d/%m/%Y %H:%M:%S'),dt.datetime.strptime('13/07/2022 00:00:00', '%d/%m/%Y %H:%M:%S')])
plt.xlabel('TriggerTime')
plt.ylabel('LAPPD offset [s]')
plt.xticks(rotation=45)
plt.savefig('plots/LAPPDOffset_TriggerTime_Area11.pdf')

list_drift.append(coeff11[0])
timestamp_drift.append(dt.datetime.strptime('03/07/2022 18:00:00', '%d/%m/%Y %H:%M:%S'))

In [ ]:
print(list_drift)
print(timestamp_drift)

list_drift_s_day=[]
for drift in list_drift:
    list_drift_s_day.append(drift*60*60*24)

In [ ]:
plt.plot(timestamp_drift,list_drift_s_day,'kd')
plt.ylabel('Drift [s/day]')
plt.xticks(rotation=45)
plt.grid()
plt.savefig('plots/LAPPD_Drift_vs_Triggertime.pdf')